## 네이버 뉴스기사 크롤링

In [1]:
# Library Import
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import sys, os
from tqdm import tqdm
from openpyxl import Workbook

In [2]:
# 저장할 엑셀 파일 생성하기
# Workbook 생성
wb = Workbook()

# Sheet 활성
sheet = wb.active

# 데이터프레임 내 header(변수명)생성
sheet.append(["title", "date", "content", "link"])

In [3]:
# 정보 입력하기
SEARCH_WORD = '스마트팜'
START_DATE = '2020.01.01'
END_DATE = '2020.12.31'
START_DATE_2 = '20200101'
END_DATE_2 = '20201231'

In [4]:
# url format 지정
url_format = 'https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}&docid=&nso=so:r,p:from{3}to{4},a:all&mynews=0&cluster_rank=34&start={5}'

In [5]:
# 안티 크롤링 방지
headers={"User-Agent": "Mozilla/5.0"}


# 페이지 크롤링
for page_number in tqdm(range(500)):
    r = requests.get(url_format.format(SEARCH_WORD, START_DATE, END_DATE, START_DATE_2, END_DATE_2, str(page_number*10+1)),  headers=headers)
    sp = BeautifulSoup(r.text, 'html.parser')
    sources = sp.select('div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info')

    # 뉴스 기사 크롤링 : 네이버 뉴스만
    naver_sources = []
    for source in sources:
        if 'https://news.naver.com/' in source.attrs['href']:
            (naver_sources).append(source.attrs['href'])
    for source in naver_sources:
        news_r = requests.get(source, headers=headers)
        news_sp = BeautifulSoup(news_r.text, 'html.parser')

        container = news_sp.select('td.content > div#main_content')
        for con in container:
            t = con.select_one("div.article_header > div.article_info > h3#articleTitle").text.strip() # title
            d = con.select_one("div.article_header > div.article_info > div.sponsor > span.t11").text.strip() # date
            c = con.select_one("div.article_body > div#articleBodyContents").text.strip() # content
            l = source # link

        # sheet 내 각 행에 데이터 추가
        sheet.append([t, d, c, l])

        # 수집한 데이터 저장
        wb.save("smartFarm_2020_new.xlsx")

100%|██████████| 500/500 [15:36<00:00,  1.87s/it]
